# Preprocessing the IPL Matches Dataset
This notebook performs basic cleaning, renaming, and feature engineering on `matches.csv` so it’s ready for loading into MySQL.

In [2]:
# 1. Import libraries and load raw data
import pandas as pd

# Load the raw matches CSV
matches_df = pd.read_csv('dataset/matches.csv')
matches_df.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [3]:
# 2. Rename columns for SQL readiness
matches_df.rename(columns={
    'id': 'match_id',
    'date': 'match_date',
    'winner': 'match_winner'
}, inplace=True)

# Verify new column names
print(matches_df.columns.tolist())

['match_id', 'season', 'city', 'match_date', 'match_type', 'player_of_match', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'match_winner', 'result', 'result_margin', 'target_runs', 'target_overs', 'super_over', 'method', 'umpire1', 'umpire2']


In [4]:
# 3. Fill missing values in text and numeric columns
matches_df.fillna({
    'city': 'Unknown',
    'player_of_match': 'None',
    'result_margin': 0,
    'target_runs': 0,
    'target_overs': 0,
    'method': 'None'
}, inplace=True)

# Confirm no nulls remain in those columns
print(matches_df[['city','player_of_match','result_margin','target_runs','target_overs','method']].isna().sum())


city               0
player_of_match    0
result_margin      0
target_runs        0
target_overs       0
method             0
dtype: int64


In [5]:
# 4. Convert match_date to datetime type
matches_df['match_date'] = pd.to_datetime(matches_df['match_date'], format='%Y-%m-%d')

# Confirm dtype
print(matches_df['match_date'].dtype)

datetime64[ns]


In [8]:
# 5. Data type conversions: ensure numeric columns are integers
for col in ['match_id', 'result_margin', 'target_runs', 'target_overs']:
    matches_df[col] = matches_df[col].astype(int)

# Confirm dtypes
print(matches_df[['match_id','season','result_margin','target_runs','target_overs']].dtypes)

match_id          int32
season           object
result_margin     int32
target_runs       int32
target_overs      int32
dtype: object


In [9]:
# 6. Convert 'super_over' flag to integer (1 if 'Y', else 0)
matches_df['super_over_flag'] = (matches_df['super_over'] == 'Y').astype(int)

# Optionally drop the original 'super_over' column:
matches_df.drop(columns=['super_over'], inplace=True)

# Confirm the new column
print(matches_df[['super_over_flag']].head())

   super_over_flag
0                0
1                0
2                0
3                0
4                0


In [10]:
# 7. Feature engineering: create 'match_result_type' from 'result'
def classify_result(row):
    if row['result'] == 'runs':
        return 'Win by Runs'
    elif row['result'] == 'wickets':
        return 'Win by Wickets'
    else:
        return 'Other'

matches_df['match_result_type'] = matches_df.apply(classify_result, axis=1)

# Show sample
print(matches_df[['result','match_result_type']].head())

    result match_result_type
0     runs       Win by Runs
1     runs       Win by Runs
2  wickets    Win by Wickets
3  wickets    Win by Wickets
4  wickets    Win by Wickets


In [11]:
# 8. (Optional) Extract Year and Month from match_date
matches_df['match_year'] = matches_df['match_date'].dt.year
matches_df['match_month'] = matches_df['match_date'].dt.month

# Show sample
print(matches_df[['match_date','match_year','match_month']].head())

  match_date  match_year  match_month
0 2008-04-18        2008            4
1 2008-04-19        2008            4
2 2008-04-19        2008            4
3 2008-04-20        2008            4
4 2008-04-20        2008            4


In [12]:
# 9. Reorder columns (primary keys first)
cols_order = [
    'match_id','season','match_date','match_year','match_month','city',
    'team1','team2','toss_winner','toss_decision','match_winner','result','match_result_type',
    'result_margin','target_runs','target_overs','super_over_flag','method',
    'player_of_match','umpire1','umpire2'
]
matches_df = matches_df[cols_order]

# Verify final column order
matches_df.head()

,match_id,season,match_date,match_year,match_month,city,team1,team2,toss_winner,toss_decision,...,result,match_result_type,result_margin,target_runs,target_overs,super_over_flag,method,player_of_match,umpire1,umpire2
0,335982,2007/08,2008-04-18,2008,4,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,runs,Win by Runs,140,223,20,0,None,BB McCullum,Asad Rauf,RE Koertzen
1,335983,2007/08,2008-04-19,2008,4,Chandigarh,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,...,runs,Win by Runs,33,241,20,0,None,MEK Hussey,MR Benson,SL Shastri
2,335984,2007/08,2008-04-19,2008,4,Delhi,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,...,wickets,Win by Wickets,9,130,20,0,None,MF Maharoof,Aleem Dar,GA Pratapkumar
3,335985,2007/08,2008-04-20,2008,4,Mumbai,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,...,wickets,Win by Wickets,5,166,20,0,None,MV Boucher,SJ Davis,DJ Harper
4,335986,2007/08,2008-04-20,2008,4,Kolkata,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,...,wickets,Win by Wickets,5,111,20,0,None,DJ Hussey,BF Bowden,K Hariharan


In [14]:
# 10. Save the cleaned DataFrame to a new CSV
matches_df.to_csv('preprocessed_data/matches_cleaned.csv', index=False)

# Display confirmation
print("matches_cleaned.csv has been written. Row count:", len(matches_df))

matches_cleaned.csv has been written. Row count: 1095


(End of preprocessing steps for `matches.csv`. Your cleaned file is now ready for import into MySQL as `matches_cleaned.csv`.)